In [17]:
import requests
from bs4 import BeautifulSoup

In [27]:
url = 'https://www.latam.com/es_co/apps/personas/booking?fecha1_dia=31&fecha1_anomes=2020-12&auAvailability=1&ida_vuelta=ida&vuelos_origen=Bogot%C3%A1&from_city1=BOG&vuelos_destino=Ciudad%20de%20M%C3%A9xico&to_city1=MEX&flex=1&vuelos_fecha_salida_ddmmaaaa=11/09/2020&cabina=Y&nadults=2&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0&application=#/'

In [28]:
# To handle 403 error
headers = {'User-Agent': 'Chrome/85.0.4183.83'}
r = requests.get(url, headers=headers)

In [29]:
r.status_code

200

In [21]:
s = BeautifulSoup(r.text,'lxml')

In [30]:
# print(s.prettify())

## Importando selenium

In [31]:
from selenium import webdriver

Instanciando el driver de Chrome

In [38]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(options=options)

In [39]:
driver.get(url)

In [36]:
driver.close()